### The code and data are adapted from:  https://medium.com/@vitalshchutski/french-nlp-entamez-le-camembert-avec-les-librairies-fast-bert-et-transformers-14e65f84c148

In [1]:
# !conda install torch
# !pip install fast-bert==1.9.1
# !mkdir model
# !mkdir finetuned_model

In [1]:
import torch
from fast_bert.data_cls import BertDataBunch 
from fast_bert.learner_cls import BertLearner
from fast_bert.data_lm import BertLMDataBunch
from fast_bert.learner_lm import BertLMLearner
from fast_bert.metrics import fbeta, roc_auc
from fast_bert.prediction import BertClassificationPredictor
from pathlib import Path
import pandas as pd
import logging



device_cuda = torch.device("cuda")
logging.basicConfig(
    handlers=[logging.FileHandler('logfile.log', 'w', 'utf-8')],
    format='%(levelname)s: %(message)s',
    datefmt='%m-%d %H:%M',
    level=logging.INFO #CRITICAL ERROR WARNING  INFO    DEBUG    NOTSET 
)
logger = logging.getLogger()

In [2]:
DATA_PATH = Path('./data/')
LOG_PATH = Path('./logs/')
MODEL_PATH = Path('./model/')
LABEL_PATH = Path('./labels/')

In [3]:
df = pd.read_csv('./data/bsv_chunk256_ba_mld_1201-1500.csv')

In [4]:
val_set = df.sample(frac=0.2, replace=False, random_state=42)
train_set = df.drop(index = val_set.index)
print('Nombre de commentaires dans le val_set:',len(val_set))
print('Nombre de commentaires dans le train_set:', len(train_set))
val_set.to_csv('./data/val_set.csv')
train_set.to_csv('./data/train_set.csv')

Nombre de commentaires dans le val_set: 860
Nombre de commentaires dans le train_set: 3441


In [5]:
labels = df.columns[1:3].to_list() 
with open('./labels/labels.txt', 'w') as f:
    for i in labels:
        f.write(i + "\n")

In [6]:
df_texts = pd.read_csv('./data/bsv_chunk256_raw_1001-1200.csv')
all_texts = df_texts['report_text'].to_list()
print('Nombre de bloc de texte:', len(all_texts))

Nombre de bloc de texte: 2801


### Création de LMDataBunch

In [7]:
databunch_lm = BertLMDataBunch.from_raw_corpus(
                    data_dir=DATA_PATH,
                    text_list=all_texts,
                    tokenizer='camembert-base',
                    batch_size_per_gpu=4, #was 16, even 8 won't do
                    max_seq_length=256, #was 512
                    multi_gpu=False,
                    model_type='camembert-base',
                    logger=logger)

### Création de LMLearner

In [64]:
lm_learner = BertLMLearner.from_pretrained_model(
                            dataBunch=databunch_lm,
                            pretrained_path='camembert-base',
                            output_dir=MODEL_PATH,
                            metrics=[],
                            device=device_cuda,
                            logger=logger,
                            multi_gpu=False,
                            logging_steps=50,
                            is_fp16=True) #was true with gpu

In [65]:
lm_learner.fit(epochs=2, #was 30
            lr=1e-4,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0


(142, 2.3500409554427777)

In [66]:
lm_learner.validate()

{'loss': 0.1194742675870657, 'perplexity': 1.1269042491912842}

In [67]:
lm_learner.save_model()

In [68]:
del lm_learner

### Création de databunch pour la classification

In [69]:
databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='camembert-base',
                          train_file='train_set.csv',
                          val_file='val_set.csv',
                          label_file='labels.txt',
                          text_col='report_text',
                          label_col=['Bioagressor','Disease'],
                          batch_size_per_gpu=8,
                          max_seq_length=256,
                          multi_gpu=False,
                          multi_label=True,
                          model_type='camembert-base')

### Création de Learner

In [70]:
metrics = [{'name': 'fbeta', 'function': fbeta}, {'name': 'roc_auc', 'function': roc_auc}]
OUTPUT_DIR = Path('./finetuned_model')
WGTS_PATH = Path('model/model_out/pytorch_model.bin')

In [71]:
# issue fast-bert pos_weight <= downgrade to 1.9.1 solve the prob
cl_learner = BertLearner.from_pretrained_model(
                        databunch,
                        pretrained_path='model/model_out',
                        metrics=metrics,
                        device=device_cuda, #was device_cuda
                        logger=logger,
                        output_dir=OUTPUT_DIR,
                        finetuned_wgts_path=WGTS_PATH,
                        warmup_steps=300,
                        multi_gpu=False,
                        multi_label=True,
                        is_fp16=True,#True when is cuda
                        logging_steps=50)

In [ ]:
cl_learner.fit(epochs=5,# was 30
            lr=2e-5,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

In [75]:
cl_learner.validate()

{'loss': 0.3738116010747574,
 'fbeta': 0.4007752239704132,
 'roc_auc': 0.9127443381413668}

In [76]:
cl_learner.save_model()

### Prédictions

In [77]:
predictor = BertClassificationPredictor(
                model_path='finetuned_model/model_out',
                label_path='labels/',
                multi_label=True,
                model_type='camembert-base',
                do_lower_case=False)

In [78]:
#cas disease: 0, bioagressor: 1 - cicadelle
predictor.predict("Cicadelles La cicadelle Edwardsiana est toujours observée sur les parcelles en été")

C:\Users\Raja\Documents\SJI\tools\anaconda3\envs\fast-camembert\lib\site-packages\pandas\core\frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


[('bioagressor', 0.93701171875), ('disease', 0.0284423828125)]

In [79]:
predictor.predict("election américane")

[('bioagressor', 0.0261993408203125), ('disease', 0.01287078857421875)]

In [80]:
#cas disease: 0, bioagressor: 0 - texte sur mouche de carotte mais pas de rique
predictor.predict("mouche de la carotte :ajouter trichloronate")

[('bioagressor', 0.849609375), ('disease', 0.021484375)]

In [81]:
#cas disease: 0, bioagressor: 0 - oïdium : pas d'intervention 
predictor.predict("oïdium : pas d'intervention dans l'immediat les conditions très chaudes et l'absence de rosées nocturnes sont défavorables à cette maladie. aucun symptôme n'a encore été observé.")

[('disease', 0.9423828125), ('bioagressor', 0.047882080078125)]

In [82]:
#cas disease: 1, bioagressor: 0 - oidium  => good
predictor.predict("oïdium du pommierce champignon est à l'heure actuelle en pleine fructification. il est nécessaire d'ajouter un antioïdium aux bouillies pour assurer'la protection du jeune feuillage.")

[('disease', 0.96240234375), ('bioagressor', 0.07196044921875)]

In [83]:
#cas disease: 1, bioagressor: 0 - mildiou
predictor.predict("vigilant : en particulier vis-à-vis du mildiou, de l’oïdium et de la bactériose / cladosporiose. Les prévisions pour les prochains jours restent peu favorables à l’expression de la bactériose et la cladosporiose, si elles se confirment. Mildiou (Pseudoperonospora cubensis) : Le modèle annonce un risque élevé pour toutes les dates de plantation avec les données de la station de Thurageau. Avec les données de la station de Maulay, les plantations en S25 et S26 montrent un risque modéré. Le risque est un peu plus élevé dans le sud de la Charente- Maritime que dans le Poitou. Niveau de risque Faible Moyen Élevé Très élevé Indice : Log (Nb de taches/unité de surface) -14 à -9 -9 à -4 -4 +4 Équivalent en unité de surface 1 tâche par hectare par 100 m2 1 tâche par 100 m2 par m2 1 tâche par m2à 1 % de surface atteinte 1 % à 100 % de surface atteinte Évaluation du risque : les conditions restent favorables à ce microorganisme (qui n’est pas un champignon, mais proche d’une algue). Les BSV sont disponibles en accès direct sur le site  (rubrique : Nos publications - Bulletin de santé du végétal) ou par abonnement en ligne gratuit sur le site  BSV CULTURES LÉGUMIÈRES DE PLEIN")

[('disease', 0.97607421875), ('bioagressor', 0.1229248046875)]

In [84]:
#cas disease: 0, bioagressor: 0 - texte sur trump trump
predictor.predict("L'avance de Donald Trump dans cet Etat où 4,9 millions d'électeurs ont voté, a fondu vendredi 6 novembre. Le candidat républicain compte")

[('bioagressor', 0.0264129638671875), ('disease', 0.01287078857421875)]

In [85]:
#cas disease: 1, bioagressor: 1 - #mildiou (removed) mais pas de risque
predictor.predict("Pomme de te r re du 17 mai 2013 BS omme Ce - N° 9 N° 9 En résumé : Risque  au 17 mai : entrée en phase de risque pour les variétés sensibles pour 5 secteurs. Pour autant, le seuil de nuisibilité n’est pas atteint. Utilisation du modèle Mileos® (www.mileos.fr) Le BSV pomme de terre de la région Centre mobilise le modèle Mileos® qui se base sur le cycle épidémique de Phytophtora infestans. • Quand démarre le risque  ? Le suivi du nombre des générations de  est un bon indicateur pour connaître le début de la période à risque de cette maladie. En fonction de la sensibilité variétale, le risque démarre : - à la sortie de")

[('disease', 0.019012451171875), ('bioagressor', 0.01861572265625)]

In [86]:
#cas disease: 0, bioagressor: 0 - zero mot clé
predictor.predict("par jour. Sur les autres secteurs, en production de carotte, les captures sont nulles. Évaluation du risque")

[('bioagressor', 0.0290374755859375), ('disease', 0.01146697998046875)]

In [87]:
#cas disease: 0, bioagressor: 0 + mildiou
predictor.predict("7 Action pilotée par le ministère chargé de l’agriculture mildiou, avec l’appui financier de l’Office National de l’Eau et des Milieux Aquatiques (ONEMA), par les crédits issus de la redevance pour pollutions diffuses attribués au")

[('disease', 0.9384765625), ('bioagressor', 0.045013427734375)]

In [88]:
#cas disease: 0, bioagressor: 0
predictor.predict("financement du plan Ecophyto. Ce bulletin est rédigé par l'ACPEL avec la collaboration de référents par culture (techniciens des Chambres d'Agriculture de la Charente, de la Charente-Maritime et d'Indre et Loire et de la Vienne) sur la base d'observations réalisées par des producteurs et techniciens : Charentes- Alliance, les entreprises de production de melon, la coopérative AGROLEG, la coopérative UNIRE, des producteurs d'Agrobio Poitou-Charentes. Ce bulletin est réalisé à partir d'observations ponctuelles. Il a pour vocation de donner une tendance de la situation sanitaire régionale. Celle-ci ne peut être transposée telle quelle dans les parcelles de production légumières (conditions très variables). La Chambre Régionale d'Agriculture de Poitou-Charentes et le rédacteur dégagent toute responsabilité quant aux décisions prises par les producteurs pour la protection de leurs cultures. Elle les invite à prendre ces décisions sur la base des observation s qu'ils auront réalisées dans leurs parcelles. Les")

[('bioagressor', 0.026611328125), ('disease', 0.01238250732421875)]

In [89]:
#cas disease: 0, bioagressor: 0 - french crop usasge
predictor.predict("Ensemble de plantes cultivées pour leurs fruits ou leurs graines riches en matières grasses (lipides). De ces fruits et graines sont extrait une huile à usage alimentaire humaine, alimentaire animal ou industriel. Les résidus de l'extraction constituent les tourteaux utilisés pour l'alimentation animale.")

[('bioagressor', 0.0259552001953125), ('disease', 0.012725830078125)]

In [90]:
#cas disease: 0, bioagressor: 0 - french crop usasge
predictor.predict("Orge semé après le 1er février, principalement de mars à mai. Orge de printemps est toujours à deux rangs")

[('bioagressor', 0.02447509765625), ('disease', 0.01363372802734375)]

In [91]:
#cas disease: 1, bioagressor: 0 - tweet - jaunisse
predictor.predict("des parcelles qui ont eu une croissance presque nulle depuis cet été en majeur partie dû à la jaunisse.")

[('disease', 0.377197265625), ('bioagressor', 0.01078033447265625)]

In [92]:
#cas disease: 1, bioagressor: 0 - tweet - sécheresse
predictor.predict("Parfois,on nous demande. Que faites-vous,par cette sécheresse?")

[('bioagressor', 0.02508544921875), ('disease', 0.01358795166015625)]

In [93]:
#cas disease: 0, bioagressor: 1 - tweet - pucerons

predictor.predict("Attention pucerons dans les blés")

[('bioagressor', 0.94482421875), ('disease', 0.03302001953125)]

### Analyse the fine-tuned model

In [94]:
predictor = BertClassificationPredictor(
                model_path='finetuned_model/model_out',
                label_path='labels/',
                multi_label=True,
                model_type='camembert-base',
                do_lower_case=False)

In [95]:
#evaluation on validation set
df_val = pd.read_csv('./data/val_set.csv')

#predictor.get_learner()

In [96]:
df_val.Disease = df_val.Disease.astype(int)
df_val.tail()

,Unnamed: 0,report_text,Bioagressor,Disease,hazard_list,source_name
855,2112,MILDIOU DU TOURNESOL (Plasmopara halstedii) No...,0,0,NaN,Note_Commune-Mildiou-Tournesol-2012_cle0b4a54
856,1146,. . . . . . . . . . . . . . . . . . . . . . . ...,0,0,NaN,draaf.centre-val-de-loire.agriculture.gouv.fr_...
857,3528,"conditions automnales et hivernales, particuli...",0,0,NaN,www.paysdelaloire.chambagri.fr_fileadmin_docum...
858,4062,teigne des crucifères (Plutella xylostella) à ...,1,0,#teigne des crucifères #mineuse,BSV_-_Legumes_no10-1_cle0f73f2
859,4246,– N ° rédigé par Hervé FRANCOIS - Chambre d’a...,0,1,#septoriose,20130430_bsv_grandescultures_13_cle4d3d84


In [97]:
batch_predictions = predictor.predict_batch(df_val.report_text.to_list())

C:\Users\Raja\Documents\SJI\tools\anaconda3\envs\fast-camembert\lib\site-packages\pandas\core\frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


In [98]:
batch_predictions[:10]

[[('bioagressor', 0.03594970703125), ('disease', 0.009674072265625)],
 [('bioagressor', 0.026702880859375), ('disease', 0.0125274658203125)],
 [('disease', 0.96826171875), ('bioagressor', 0.9609375)],
 [('bioagressor', 0.984375), ('disease', 0.763671875)],
 [('bioagressor', 0.93896484375), ('disease', 0.0290374755859375)],
 [('bioagressor', 0.024139404296875), ('disease', 0.0136871337890625)],
 [('bioagressor', 0.95947265625), ('disease', 0.04986572265625)],
 [('bioagressor', 0.027740478515625), ('disease', 0.0119171142578125)],
 [('bioagressor', 0.96630859375), ('disease', 0.10992431640625)],
 [('bioagressor', 0.06805419921875), ('disease', 0.0070648193359375)]]

In [99]:
#dict(batch_predictions[1]).values()
list_y_pred = [ dict(pred) for pred in batch_predictions]
#list_y_pred[-5:]

In [100]:
df_y_pred = pd.DataFrame(list_y_pred, columns =['bioagressor', 'disease']) 
df_y_pred = df_y_pred.rename(columns={"bioagressor": "Bioagressor", "disease": "Disease"})
df_y_pred.tail()

,Bioagressor,Disease
855,0.065491,0.959473
856,0.026962,0.012238
857,0.029587,0.890137
858,0.874023,0.015961
859,0.110474,0.973633


In [101]:
df_y_pred.describe()

,Bioagressor,Disease
count,860.000000,860.000000
mean,0.454651,0.282911
std,0.440345,0.405159
min,0.010094,0.005260
25%,0.027103,0.012573
50%,0.166199,0.025398
75%,0.943481,0.829590
max,0.984863,0.988770


In [102]:
df_y_real = pd.DataFrame(df_val, columns=['Bioagressor', 'Disease'])
df_y_real.tail()

,Bioagressor,Disease
855,0,0
856,0,0
857,0,0
858,1,0
859,0,1


In [103]:
df_y_real.describe()

,Bioagressor,Disease
count,860.000000,860.000000
mean,0.366279,0.201163
std,0.482067,0.401103
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,1.000000,0.000000
max,1.000000,1.000000


In [104]:
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

In [105]:
precision_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average=None) #If None, the scores for each class are returned

array([0.73433584, 0.65447154])

In [106]:
precision_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5, average='weighted')

0.7060232924374109

In [107]:
recall_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average=None)

array([0.93015873, 0.93063584])

In [108]:
recall_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average='weighted')

0.930327868852459

In [109]:
f1_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average=None)

array([0.82072829, 0.76849642])

In [110]:
f1_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average='weighted')

0.8022116648216467

In [111]:
accuracy_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5)

0.7732558139534884

In [112]:
accuracy_score(y_true=df_y_real.Bioagressor.values, y_pred=df_y_pred.Bioagressor.values > 0.5)


0.8511627906976744

In [113]:
accuracy_score(y_true=df_y_real.Disease.values, y_pred=df_y_pred.Disease.values > 0.5 )


0.8872093023255814

In [114]:
from sklearn.metrics import average_precision_score
average_precision_score(df_y_real.values, df_y_pred.values, average=None)

array([0.79783766, 0.69907958])

In [115]:
from sklearn.metrics import average_precision_score
average_precision_score(df_y_real.values, df_y_pred.values, average='weighted')

0.7628271090985996

In [116]:
df_results = pd.concat([df_val, df_y_pred > 0.5], axis=1, ignore_index=True)

In [117]:
df_results.to_csv('predictions_camembert-base.csv')